### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [8]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [22]:
num_iterations = 1
empty_list = []
for i in range(1, 42):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.99


######### Lightcurve features #########

amplitude = 0.12487061573345674
cusum = 0.08258719960605884
eta = 1.9287135516370633
hl_amp_ratio = 0.5358786853951728
kurtosis = -0.09052618958499847
n_points = 636
period = 0.48509106463129903
period_SNR = 148.22418823773333
period_log10FAP = -74.24033958648745
period_uncertainty = 0.0001024912500075803
phase_cusum = 0.19472283760776954
phase_eta = 0.7482064183090786
phi21 = -2.6167382621807445
phi31 = -1.9368241247541276
quartile31 = 0.14825516334237854
r21 = 0.3894729420708466
r31 = 0.17091037709116327
shapiro_w = 0.9546677470207214
skewness = -0.7039411912427861
slope_per10 = -0.06760244089582325
slope_per90 = 0.05694822113824875
stetson_k = 0.7313989104166629
weighted_mean = 13.923158857840692
weighted_std = 0.10852394132998275
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4747464420290012
cusum = 0.31388509556004357
eta = 1.8795960401353526
hl_amp_ratio = 0.32520511703288224
kurtosis = -0.4638756299359761
n_points = 461
period = 0.46197284192571736
period_SNR = 117.50460599692687
period_log10FAP = -58.34182778213228
period_uncertainty = 9.267258987280536e-05
phase_cusum = 0.3275902971797354
phase_eta = 0.4021126193110886
phi21 = 0.5409557913973274
phi31 = 1.3941550282334765
quartile31 = 0.6186348062660407
r21 = 0.40221436386627
r31 = 0.2570103223140407
shapiro_w = 0.911858856678009
skewness = -0.7758487344732041
slope_per10 = -0.032868127289816806
slope_per90 = 0.035429323790953336
stetson_k = 0.631192564069206
weighted_mean = 15.88812080428445
weighted_std = 0.4324614861872479
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.9


######### Lightcurve features #########

amplitude = 0.4445363176571876
cusum = 0.3144238247448124
eta = 1.8861619912392809
hl_amp_ratio = 0.574067519774967
kurtosis = -0.08456183882079404
n_points = 1941
period = 0.4506927197806037
period_SNR = 215.8475550503019
period_log10FAP = -159.167723922293
period_uncertainty = 7.311692588718444e-05
phase_cusum = 0.33544361267443873
phase_eta = 1.0069980088659507
phi21 = -2.5854876477198037
phi31 = -1.7001370881948792
quartile31 = 0.5911169422495526
r21 = 0.44747618641805326
r31 = 0.2648143555642682
shapiro_w = 0.984309196472168
skewness = -0.21732632530655496
slope_per10 = -0.005069277606989824
slope_per90 = 0.005202923322624271
stetson_k = 0.6455621748367536
weighted_mean = 15.266157314685945
weighted_std = 0.5025399134359775
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.69


######### Lightcurve features #########

amplitude = 0.21903260000625568
cusum = 0.12922492412562775
eta = 1.4234565354934583
hl_amp_ratio = 0.9101940609459164
kurtosis = -1.396758343294538
n_points = 1379
period = 0.35013826682239474
period_SNR = 215.4094760492033
period_log10FAP = -257.6490675220078
period_uncertainty = 4.396071114420863e-05
phase_cusum = 0.330479275626412
phase_eta = 0.19268126282982306
phi21 = 1.2551907229700006
phi31 = 3.323775042105968
quartile31 = 0.32564342813227576
r21 = 0.10824689728308279
r31 = 0.0523488337298867
shapiro_w = 0.9300854206085205
skewness = -0.034912973613114566
slope_per10 = -0.02773867418818365
slope_per90 = 0.024904508559476426
stetson_k = 0.8424604883170156
weighted_mean = 13.837253869755164
weighted_std = 0.15803077902054372


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
